In [2]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-18 15:27:51--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: 'minsearch.py'

     0K ...                                                   100% 2.25M=0.002s

2024-06-18 15:27:52 (2.25 MB/s) - 'minsearch.py' saved [3832/3832]



In [2]:
import json
with open ('./documents.json', 'rt') as txt:
    doc_raw = json.load(txt)

In [3]:
# doc_raw

In [33]:
documnents = []
for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documnents.append(doc)

In [34]:
documnents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Elastic search

In [6]:
# !pip install elasticsearch

## Q1

In [22]:
from elasticsearch import Elasticsearch

In [23]:
es = Elasticsearch("http://localhost:9200/")
es.info()

ObjectApiResponse({'name': '3fd1e3eb7029', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'qLpCz4Q3R9GdOizpS7-XCQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [24]:
index_setting ={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

## Q2

In [29]:
index_name = "course-questions_1"
response = es.indices.create(index=index_name, body=index_setting)
# response = es.indices.get(index=index_name)
response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions_1'})

In [39]:
from tqdm.auto import tqdm

for doc in tqdm(documnents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [40]:
user_question = "How do I execute a command in a running docker container?"



## Q3

In [62]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text" ],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [85]:
response = es.search(index=index_name, body=search_query)

In [87]:
search_query

{'size': 5,
 'query': {'bool': {'must': {'multi_match': {'query': 'How do I execute a command in a running docker container?',
     'fields': ['question^4', 'text'],
     'type': 'best_fields'}},
   'filter': {'term': {'course': 'machine-learning-zoomcamp'}}}}}

In [64]:
response

ObjectApiResponse({'took': 36, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions_1', '_id': 'H4MXbpABvoDpS-TgVp_1', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions_1', '_id': 'PoMXbpABvoDpS-TgXp-J', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\

In [65]:
response['hits']

{'total': {'value': 345, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions_1',
   '_id': 'H4MXbpABvoDpS-TgVp_1',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions_1',
   '_id': 'PoMXbpABvoDpS-TgXp-J',
   '_score': 51.04628,
   '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can 

## Q4

In [67]:
for hit in response['hits']['hits']:
    doc = hit["_source"]
    print (f"Section: {doc['section']} \nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Section: 5. Deploying Machine Learning Models 
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Section: 5. Deploying Machine Learning Models 
Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani


Section: 5. Deploying Machine Learning Models 
Question: How do I copy files from a

In [106]:
q1 = "How do I execute a command in a running docker container?"

In [107]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": q1,
                    "fields": ["question^4", "text" ],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [108]:
def retrieve_contex(query):
    return es.search(index=index_name, body=query)

cont = retrieve_contex(query=search_query)
 
context = ""
for hit in cont['hits']['hits']:
    doc = hit["_source"]
    context += f"Section: {doc['section']} \nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"

print (context)
    

Section: 5. Deploying Machine Learning Models 
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Section: 5. Deploying Machine Learning Models 
Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Section: 5. Deploying Machine Learning Models 
Question: How do I copy files from a d

In [117]:
question = "How do I execute a command in a running docker container?"
def retrieve_contex(query):
    return es.search(index=index_name, body=query)

cont = retrieve_contex(query=search_query)
 
context = ""
for hit in cont['hits']['hits']:
    doc = hit["_source"]
    # context += f"Section: {doc['section']} \nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    context += f"{doc['text']}\n"

print (context)

Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)
You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani
You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:
COPY ["src/predict.py", "models/xgb_model.bin", "./"]											Gopakum

In [118]:
question = q1
text = context

In [119]:
context_template = f"""
Q: {question}\n\n
A: {text}
""".strip()

In [122]:
len(context_template)

2154

In [123]:
question = "How do I execute a command in a running docker container?"

In [124]:
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [4]:
# print(prompt_template.strip())

## Q5

In [128]:
len(prompt_template)

2332

In [132]:
# !pip install tiktoken

In [130]:
import tiktoken

In [131]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [133]:
num_of_tokens =encoding.encode(prompt_template)

## Q6

In [135]:
len(num_of_tokens)

529